<a href="https://colab.research.google.com/github/RVP20/STProject/blob/main/Stereoscope2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install loompy

     |████████████████████████████████| 41 kB 108 kB/s 
     |████████████████████████████████| 48 kB 5.6 MB/s 
  Created wheel for loompy: filename=loompy-3.0.6-py3-none-any.whl size=47906 sha256=e39708d0a22d99a0c65524d63f0f3cb1315fa214d435e869639ac67078f2e288
  Stored in directory: /root/.cache/pip/wheels/aa/84/e5/38fa2e3b59faa40b027a12a0c2de64cd787a1cbf05725fb021
  Created wheel for numpy-groupies: filename=numpy_groupies-0.9.14-py3-none-any.whl size=24113 sha256=1ea4b920d179b4a942c846f83de018b49abf905af061c79d02ea68206ab68dc6
  Stored in directory: /root/.cache/pip/wheels/e7/68/13/d9f5f7dd1a3aacf8e9c2c12c6e86cceed1c628f4ae60ff5b01
Successfully built loompy numpy-groupies


In [3]:
%%shell
git clone https://github.com/almaan/stereoscope 
cd stereoscope
./setup.py install

Cloning into 'stereoscope'...
remote: Enumerating objects: 612, done.
remote: Counting objects: 100% (304/304), done.
remote: Compressing objects: 100% (222/222), done.
remote: Total 612 (delta 169), reused 202 (delta 79), pack-reused 308
Receiving objects: 100% (612/612), 57.61 MiB | 15.73 MiB/s, done.
Resolving deltas: 100% (322/322), done.
running install
running bdist_egg
running egg_info
creating stereoscope.egg-info
writing stereoscope.egg-info/PKG-INFO
writing dependency_links to stereoscope.egg-info/dependency_links.txt
writing entry points to stereoscope.egg-info/entry_points.txt
writing requirements to stereoscope.egg-info/requires.txt
writing top-level names to stereoscope.egg-info/top_level.txt
writing manifest file 'stereoscope.egg-info/SOURCES.txt'
adding license file 'LICENSE.txt'
writing manifest file 'stereoscope.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating 

In [4]:
%%shell
stereoscope test

In [5]:
import os.path as osp
import os
import datetime
import sys
import re

import numpy as np
import pandas as pd
import loompy as lp

import argparse as arp

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


In [6]:
#pth = sys.argv[1]
#odir = sys.argv[2]

ds = lp.connect("./gdrive/MyDrive/ST_Project/l1_hippocampus.loom","r")

celltype_1 = np.array([ '_'.join([str(x).replace(',','_'),str(y)]) for \
                       x,y in zip(ds.ca['Class'],ds.ca['Clusters']) ])

celltype_2 = np.array([ '_'.join([str(x).replace(',','_'),str(y)]) for \
                       x,y in zip(ds.ca['Subclass'],ds.ca['Clusters']) ])


new_ds_ca = dict(CellID = ds.ca['CellID'],
                 Clusters = ds.ca['Clusters'],
                 Subclass = ds.ca['Subclass'],
                 Celltype_1 = celltype_1,
                 Celltype_2 = celltype_2)

new_ds_ra = dict(Gene = ds.ra['Gene'])
#new_basename = '_'.join(['mod',osp.basename(pth)])
#filename = os.path.join(odir,new_basename)
lp.create("modified.loom", ds[:,:], new_ds_ra, new_ds_ca)
print("successfully created modifed loom-file")

successfully created modifed loom-file


In [7]:
#!/usr/bin/env python3

prs = arp.ArgumentParser()
'''
prs.add_argument('-lf','--loompy_file',
                 required = True,
                 type = str,
                 help = ' '.join(['Full path to loompy',
                                  'file.',
                                 ]))

#prs.add_argument('-o','--output_dir',
                 required = False,
                 type = str,
                 default = '.',
                 help = ' '.join(['Path to output directory.',
                                  'If not specified curret work',
                                  'directory is used',
                                 ]))

#prs.add_argument('-lb','--lower_bound',
                 required = False,
                 type = int,
                 default = 0,
                 help = ' '.join(['Lower bound for number',
                                  'of cells to be present',
                                  'in a cell type.',
                                  'Types with less than specified',
                                  'value will be omitted from set',
                                 ]))

#prs.add_argument('-ub','--upper_bound',
                 required = False,
                 type = int,
                 default = None,
                 help = ' '.join(['Upper bound for number',
                                  'of cells to be included',
                                  'from a cell type. Types'
                                  'with more cells than specified'
                                  'value will be randomly',
                                  'subsampled',
                                 ]))
'''
'''prs.add_argument('-cn','--column_name',
    required = False,
    type = str,
    default = 'Subtype',
    help = ' '.join(['Name of column in loompy',
                                  'file which specifies',
                                  'type label',
                                 ]))


prs.add_argument('-af','--add_filter',
    required = False,
    type = str,
    nargs = 2,
    default = [None,None],
    help = ' '.join(['Additional filtering',
                                  'Column as first argument',
                                  'value as second',
                                 ]))
'''


#args = prs.parse_args()


np.random.seed(1337)

#loom_pth = args.loompy_file
lower_bound = 25
upper_bound = 50
#out_dir = args.output_dir

#if not osp.exists(out_dir):
#    os.mkdir(out_dir)

label = 'Celltype_1'

tag = str(datetime.datetime.now())
tag = re.sub('-| |:|\\.','',tag)

print(f"Unique Identifier for set >> {tag}")

try:
    ds = lp.connect("modified.loom")
except OSError:
    print(f'The file {"modified.loom"} either does not exist or',
          f" you might already have established a connection",
          f" to the file.")
    sys.exit(-1)

'''if args.add_filter[0] is not None:
    add_filter = ds.ca[args.add_filter[0]] == args.add_filter[1]
else:
    add_filter = np.ones(ds.ca[label].shape[0]).astype(np.bool)'''

add_filter = np.ones(ds.ca[label].shape[0]).astype(np.bool)
uni_types = np.unique(ds.ca[label].flatten())

use_cells = []

for type in uni_types:
    zidx = np.where((ds.ca[label].flatten() == type) & add_filter)[0]
    nz = zidx.shape[0]

    if nz < lower_bound:
        print(f"{type} | was discarded due to insufficient number of cells")
        continue

    elif nz >= upper_bound:
        zidx = np.random.choice(zidx,
                                size = upper_bound,
                                replace = False,
                                )

    fromz = zidx.shape[0]
    use_cells +=  zidx.tolist()

    print(f"{type} | Used {fromz} cells ")

use_cells = np.array(use_cells)
use_cells = np.sort(use_cells)


new_cnt = ds[:,use_cells].T
new_lbl = ds.ca[label].flatten()[use_cells]
new_barcodes = ds.ca['CellID'][use_cells]
_, idx = np.unique(new_barcodes,return_index = True)
new_cnt = new_cnt[idx,:]
new_lbl = new_lbl[idx]
new_barcodes = new_barcodes[idx]
genes = ds.ra['Gene']

new_cnt = pd.DataFrame(new_cnt,
                       index = pd.Index(new_barcodes),
                       columns = pd.Index(genes),
                      )

new_mta = pd.DataFrame(new_lbl,
                       index = pd.Index(new_barcodes),
                       columns = pd.Index(['bio_celltype']),
                       )


name,counts = np.unique(new_lbl,return_counts = True)

stats = pd.DataFrame(counts,
                     index = pd.Index(name),
                     columns = pd.Index(['members']),
                     )

print(stats)
print(f"Assembled Count Matrix with dimension >> nrow : {new_cnt.shape[0]} | ncol :  {new_cnt.shape[1]}")
print(f"Assembled Meta File with >> nrow : {new_mta.shape[0]}")

'''opth_cnt = osp.join(out_dir,
                    '.'.join([tag,
                             'cnt_data.tsv',
                            ]
                           )
                   )


opth_mta = osp.join(out_dir,
                    '.'.join([tag,
                             'mta_data.tsv',
                            ]
                           )
                   )

opth_sts = osp.join(out_dir,
                    '.'.join([tag,
                             'stats.tsv',
                            ]
                           )
                   )'''
opth_cnt='cnt_data.tsv'
opth_mta='mta_data.tsv'
opth_sts='stats.tsv'                   


new_cnt.to_csv(opth_cnt,
               sep = '\t',
               header = True,
               index = True,
               index_label = 'cell',
              )

new_mta.to_csv(opth_mta,
               sep = '\t',
               header = True,
               index = True,
               index_label = 'cell',
              )

stats.to_csv(opth_sts,
             sep = '\t',
             header = True,
             index = True,
             index_label = 'cell',
              )
ds.close()

Unique Identifier for set >> 20220303193835922362
Astrocytes_13 | was discarded due to insufficient number of cells
Astrocytes_14 | Used 50 cells 
Astrocytes_38 | was discarded due to insufficient number of cells
Astrocytes_39 | was discarded due to insufficient number of cells
Astrocytes_40 | Used 50 cells 
Astrocytes_41 | Used 31 cells 
Astrocytes_42 | Used 50 cells 
Astrocytes_44 | Used 41 cells 
Astrocytes_45 | was discarded due to insufficient number of cells
Astrocytes_48 | was discarded due to insufficient number of cells
Astrocytes_51 | was discarded due to insufficient number of cells
Astrocytes_6 | was discarded due to insufficient number of cells
Astrocytes_66 | was discarded due to insufficient number of cells
Blood_14 | was discarded due to insufficient number of cells
Blood_42 | was discarded due to insufficient number of cells
Blood_5 | was discarded due to insufficient number of cells
Blood_51 | was discarded due to insufficient number of cells
Blood_59 | was discarded 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:99: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Neurons_23 | Used 50 cells 
Neurons_24 | Used 50 cells 
Neurons_25 | Used 50 cells 
Neurons_26 | Used 50 cells 
Neurons_27 | Used 50 cells 
Neurons_28 | Used 50 cells 
Neurons_29 | was discarded due to insufficient number of cells
Neurons_3 | was discarded due to insufficient number of cells
Neurons_30 | Used 26 cells 
Neurons_31 | was discarded due to insufficient number of cells
Neurons_38 | was discarded due to insufficient number of cells
Neurons_39 | was discarded due to insufficient number of cells
Neurons_4 | was discarded due to insufficient number of cells
Neurons_40 | was discarded due to insufficient number of cells
Neurons_42 | was discarded due to insufficient number of cells
Neurons_43 | was discarded due to insufficient number of cells
Neurons_44 | was discarded due to insufficient number of cells
Neurons_45 | was discarded due to insufficient number of cells
Neurons_48 | Used 50 cells 
Neurons_49 | Used 25 cells 
Neurons_5 | was discarded due to insufficient number of c

In [8]:
!stereoscope run --sc_cnt cnt_data.tsv --sc_labels mta_data.tsv -sce 2000  -o hippo_1 -n 5000 --st_cnt ./gdrive/MyDrive/ST_Project/st-hippo*tsv -ste 2000 --gpu -stb 100 -scb 100 

[2022-03-03 19:40:40,377 - stsc - INFO ] >> Using device cuda
[2022-03-03 19:40:40,377 - stsc - INFO ] >> fitting sc data | count file : cnt_data.tsv | labels file : mta_data.tsv
[2022-03-03 19:41:19,036 - stsc - INFO ] >> SC data GENES : 5000  SC data CELLS : 2533  SC data TYPES : 56 


Epoch : 2000/2000 | Loss : 8.515567E+06 | [ =================== ] 

[2022-03-03 19:56:34,931 - stsc - INFO ] >> fit st data section(s) : ['./gdrive/MyDrive/ST_Project/st-hippo1.tsv', './gdrive/MyDrive/ST_Project/st-hippo2.tsv']
[2022-03-03 19:56:50,595 - stsc - INFO ] >> ST data GENES : 5000  ST data SPOTS : 1213 


Epoch : 2000/2000 | Loss : 7.968603E+06 | [ =================== ] 

[2022-03-03 19:59:33,154 - stsc - INFO ] >> saving proportions for section st-hippo1 to hippo_1/st-hippo1/W.2022-03-03194040.299779.tsv
[2022-03-03 19:59:33,189 - stsc - INFO ] >> saving proportions for section st-hippo2 to hippo_1/st-hippo2/W.2022-03-03194040.299779.tsv


In [10]:
!stereoscope look -pp ./hippo_1/st-hippo*/W*tsv -o viz -sc i -sb s -nc 7 -c "umap" -g -ms 40

findfont: Font family ['calibri'] not found. Falling back to DejaVu Sans.
findfont: Font family ['calibri'] not found. Falling back to DejaVu Sans.


In [11]:
!zip -r ./Results.zip ./Result

  adding: Result/ (stored 0%)
  adding: Result/hippo_1/ (stored 0%)
  adding: Result/hippo_1/R.2022-03-03194040.299779.tsv (deflated 57%)
  adding: Result/hippo_1/sc_loss.2022-03-03194040.299779.txt (deflated 65%)
  adding: Result/hippo_1/logits.2022-03-03194040.299779.tsv (deflated 50%)
  adding: Result/hippo_1/st_loss.2022-03-03194040.299779.txt (deflated 63%)
  adding: Result/hippo_1/stsc.2022-03-03194040.299779.log (deflated 60%)
  adding: Result/hippo_1/sc_model.2022-03-03194040.299779.pt (deflated 10%)
  adding: Result/hippo_1/st-hippo2/ (stored 0%)
  adding: Result/hippo_1/st-hippo2/W.2022-03-03194040.299779.tsv (deflated 59%)
  adding: Result/hippo_1/st-hippo1/ (stored 0%)
  adding: Result/hippo_1/st-hippo1/W.2022-03-03194040.299779.tsv (deflated 59%)
  adding: Result/hippo_1/st_model.2022-03-03194040.299779.pt (deflated 8%)
  adding: Result/stats.tsv (deflated 69%)
  adding: Result/modified.loom (deflated 16%)
  adding: Result/cnt_data.tsv (deflated 97%)
  adding: Result/viz/ 